<a href="https://colab.research.google.com/github/AaronTYT/IOD_capstone_GeoPPE_Sentinel/blob/main/capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project:
## GeoPPE Sentinel: AI-Powered PPE & Fatigue Safety for WA MineSites

PPE pictures dataset (Roboflow/Kaggle):

APIs used for this project:
- API 1
- API 2

#### Make sure that your VS Code is connected to Google Colab
Need Google Colab for GPU to train CNN model test this before importing all the python libraries:

Ensure that the kernel you selected is from Google Colab and ensure you selected GPU (T4)

In [1]:
import tensorflow as tf
import torch

In [2]:
print(tf.config.list_physical_devices('GPU'))
print(torch.cuda.is_available())

[]
False


In [3]:
import sys

def is_running_in_colab():
    return 'google.colab' in sys.modules

print("Running in Google Colab?", is_running_in_colab())

Running in Google Colab? True


#### Import Python libraries here

In [ ]:
# === Core scientific stack ===
import numpy as np
import pandas as pd
from scipy import stats

# === Plotting ===
import matplotlib.pyplot as plt
import seaborn as sns

# === Utilities / IO ===
import requests
from IPython.display import display, clear_output

from sklearn.calibration import CalibratedClassifierCV

# === Scikit-learn ===
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SequentialFeatureSelector

# === Models packages ===
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.base import clone

# === Models packages (ensemble) ===
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    BaggingClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
)

from sklearn.model_selection import GridSearchCV, StratifiedKFold

# === Evaulation ML ===
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,precision_recall_fscore_support,
    confusion_matrix, roc_curve, auc, roc_auc_score,
    balanced_accuracy_score, precision_score, recall_score
)
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, label_binarize
from sklearn.impute import SimpleImputer

import pickle, json, time, platform, sklearn

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

### Dataset sources
1. PPE Detection Dataset (Hardhat + Vest).
Construction PPE dataset — used as a baseline for mining PPE detection.
Resource: https://universe.roboflow.com/roboflow-universe-projects/personal-protective-equipment-combined-model/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true


2. Fatigue Detection Dataset
3. WA Mines Geospatial Dataset

In [4]:
from google.colab import drive

# Mount Google Drive at /content/drive
drive.mount('/content/drive')




Mounted at /content/drive


In [5]:
!ls "/content/drive/MyDrive/PPE_Dataset"

'Personal Protective Equipment - Combined Model.v1i.yolov8.zip'


In [6]:
!cp "/content/drive/MyDrive/PPE_Dataset/Personal Protective Equipment - Combined Model.v1i.yolov8.zip" "/content/ppe.zip"

In [7]:
!unzip /content/ppe.zip -d /content/ppe_dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/ppe_dataset/valid/labels/Nhh_img_81_jpg.rf.bae67d040350996ff947b7e479fbb834.txt  
 extracting: /content/ppe_dataset/valid/labels/OR-02-01-0743-26_l_jpg.rf.684b850bdc98dc061282939fa0c2a68a.txt  
  inflating: /content/ppe_dataset/valid/labels/PP02img1021_jpg.rf.3d994ca795873470ba054fb7ca965a07.txt  
  inflating: /content/ppe_dataset/valid/labels/PP02img1022_jpg.rf.e1c53c085c4724e6395357ecc981901d.txt  
  inflating: /content/ppe_dataset/valid/labels/PP02img1030_jpg.rf.9ca62394750c3257d22d218e128abe63.txt  
  inflating: /content/ppe_dataset/valid/labels/PP02img1035_jpg.rf.8376aa0afee4004771a8b4057a913877.txt  
  inflating: /content/ppe_dataset/valid/labels/PP02img1037_jpg.rf.0e129fc3877bd1dd10f2538c49bafc15.txt  
  inflating: /content/ppe_dataset/valid/labels/PP02img1045_jpg.rf.a2a71bac35641bc4627e123ffb7e1df4.txt  
  inflating: /content/ppe_dataset/valid/labels/PP02img1047_jpg.rf.1823b4930a7c4320453118091dd5be6f.txt 

In [10]:
!ls "ppe_dataset"

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid
